#### imports 

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
from IPython.display import clear_output,display

In [2]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()
import random

import os
from vexa_client import VexaClient

import os
TOKEN = os.getenv('ADMIN_API_TOKEN')
url = "http://localhost:18056"
admin_client = VexaClient(
    base_url=url,
    admin_key=TOKEN  # From docker-compose.yml
)

import time

def request_bot(meeting_url,language=None):
    native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"
    meeting_info = client.request_bot(
        platform="google_meet",
        native_meeting_id=native_meeting_id,
        bot_name="Vexa",
        language=language,
        task = 'transcribe'
    )
    return meeting_info
def get_transcript(meeting_url):
    native_meeting_id = meeting_url.split("/")[-1]
    try:
        for _ in range(10):
            
                transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
                df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
                clear_output()
                display(df.sort_values('absolute_start_time').tail(10))
                time.sleep(1)

    except Exception as e:
        print(e)

def stop_bot(meeting_url):
    native_meeting_id = meeting_url.split("/")[-1]
    client.stop_bot(platform="google_meet",native_meeting_id=native_meeting_id)


#### agenda

#### single bot per user/meeting
#### concurrent_bots
1. create user with 2 concurrency
2. start 1 bot
3. request another bot for the same meeting immediately -> must fail
4. first bot -> must come, admit the bot
5. speak and test transcription
6. launch bot for another meeting, admit the bot, check trnascription
7. launch another bot for the third meeting -> must fail
8. stop bot meeting 2 and and launch bot 3, but come

#### stop bot works for requested and active meetings
1 start meeting 1, stop immediately, start meeting 2 ->bot must come


#### tests

In [3]:
meeting_urls = ["https://meet.google.com/xdy-txjo-fyh", 
            "https://meet.google.com/fjp-cbnn-hug",
            "https://meet.google.com/wjf-bhmw-bix"]

##### 1. create user with 2 concurrency

In [4]:

new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", 
                                    name="test",
                                    max_concurrent_bots=2)
user_id = new_user['id']
user_id
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']
client = VexaClient(
    base_url=url,
    api_key=user_api_key)


DEBUG: Making POST request to http://localhost:18056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '59748@example.com', 'name': 'test', 'max_concurrent_bots': 2}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:08:31 GMT, Wed, 03 Sep 2025 11:08:31 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '145', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"59748@example.com","name":"test","image_url":null,"max_concurrent_bots":2,"data":{},"id":111,"created_at":"2025-09-03T11:08:31.943329"}...

DEBUG: Making POST request to http://localhost:18056/admin/users/111/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:08:31 GMT, Wed, 03 Sep 2025 11:08:31 GMT', 'server': 'uvicorn, uvic

In [5]:
#2. start 1 bot
request_bot(meeting_urls[0])
#3. request another bot for the same meeting immediately (must fail)
try:request_bot(meeting_urls[0])
except Exception as e:
    print(e) #An active or requested meeting already exists for this platform and meeting ID


DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 's6SWLYY0wuniAHVIoHpJoYy9ccvdZbdI5xyolkVu'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'xdy-txjo-fyh', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:08:51 GMT, Wed, 03 Sep 2025 11:08:51 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '384', 'content-type': 'application/json'}
DEBUG: Response content: {"id":236,"user_id":111,"platform":"google_meet","native_meeting_id":"xdy-txjo-fyh","constructed_meeting_url":"https://meet.google.com/xdy-txjo-fyh","status":"requested","bot_container_id":"63e19181b1aa060c797e9d55cb198e703c91277719488aa426d57d6ee1a646a5","start_time":null,"end_time":null,"data":{},"created_at":"2025-09-03T11:08:52.486350","updated_at":"2025-09-03T11:08:52.494181"}...

DEBUG: Making POST request to http://localhost:18056/bo

4. first bot must come, admit the bot and speak

In [6]:
time.sleep(15)
#5. speak and test transcription
get_transcript(meeting_urls[0])

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,0.00,4.26,"seven, eight, nine, 10, 11, 12, 13, 14, 15, 17,",en,None,Dmitriy Grankin,2025-09-03T11:09:11.414078+00:00,2025-09-03T11:09:15.674078+00:00
1,4.26,10.26,"first bot is here, first bot is here, we are picking up the first bot",en,None,Dmitriy Grankin,2025-09-03T11:09:15.674078+00:00,2025-09-03T11:09:21.674078+00:00


In [7]:
#6. launch bot for another meeting, admit the bot, check trnascription
request_bot(meeting_urls[1])



DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 's6SWLYY0wuniAHVIoHpJoYy9ccvdZbdI5xyolkVu'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'fjp-cbnn-hug', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:09:25 GMT, Wed, 03 Sep 2025 11:09:25 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '384', 'content-type': 'application/json'}
DEBUG: Response content: {"id":237,"user_id":111,"platform":"google_meet","native_meeting_id":"fjp-cbnn-hug","constructed_meeting_url":"https://meet.google.com/fjp-cbnn-hug","status":"requested","bot_container_id":"93002c8c4d4d94908954a6ee9b07061eb0481a2cf4c887531fd623fba32f840a","start_time":null,"end_time":null,"data":{},"created_at":"2025-09-03T11:09:25.662632","updated_at":"2025-09-03T11:09:25.670266"}...


{'id': 237,
 'user_id': 111,
 'platform': 'google_meet',
 'native_meeting_id': 'fjp-cbnn-hug',
 'constructed_meeting_url': 'https://meet.google.com/fjp-cbnn-hug',
 'status': 'requested',
 'bot_container_id': '93002c8c4d4d94908954a6ee9b07061eb0481a2cf4c887531fd623fba32f840a',
 'start_time': None,
 'end_time': None,
 'data': {},
 'created_at': '2025-09-03T11:09:25.662632',
 'updated_at': '2025-09-03T11:09:25.670266'}

In [8]:
time.sleep(15)
get_transcript(meeting_urls[1])

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,0.00,1.00,code.,en,None,Dmitriy Grankin,2025-09-03T11:09:49.662618+00:00,2025-09-03T11:09:50.662618+00:00
1,1.00,10.92,transcript for the book number two for the book number two and we delete the transcripts right here,en,None,Dmitriy Grankin,2025-09-03T11:09:50.662618+00:00,2025-09-03T11:10:00.582618+00:00


In [9]:
#7. launch another bot for the third meeting -> must fail
try:request_bot(meeting_urls[2])
except Exception as e:
    print(e) #User has reached the maximum concurrent bot limit (2).


DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 's6SWLYY0wuniAHVIoHpJoYy9ccvdZbdI5xyolkVu'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'wjf-bhmw-bix', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 403
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:10:04 GMT, Wed, 03 Sep 2025 11:10:04 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '67', 'content-type': 'application/json'}
DEBUG: Response content: {"detail":"User has reached the maximum concurrent bot limit (2)."}...
HTTP Error 403 for POST http://localhost:18056/bots: User has reached the maximum concurrent bot limit (2).


In [10]:
#8. stop bot meeting 2 and and launch bot 3, but come
stop_bot(meeting_urls[1])
request_bot(meeting_urls[2])


DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/fjp-cbnn-hug
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 's6SWLYY0wuniAHVIoHpJoYy9ccvdZbdI5xyolkVu'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:10:13 GMT, Wed, 03 Sep 2025 11:10:13 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '59', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted and is being processed."}...

DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 's6SWLYY0wuniAHVIoHpJoYy9ccvdZbdI5xyolkVu'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'wjf-bhmw-bix', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:10:13 GMT, Wed, 03 Sep 2025 11:10:13 GMT', 'server': 'uvicor

{'id': 238,
 'user_id': 111,
 'platform': 'google_meet',
 'native_meeting_id': 'wjf-bhmw-bix',
 'constructed_meeting_url': 'https://meet.google.com/wjf-bhmw-bix',
 'status': 'requested',
 'bot_container_id': '3c933c2b88656d9412d38a4fa5d1f122291a053de955e1a0fa016d70aa3addfb',
 'start_time': None,
 'end_time': None,
 'data': {},
 'created_at': '2025-09-03T11:10:13.968080',
 'updated_at': '2025-09-03T11:10:13.975518'}

In [13]:
time.sleep(15)
get_transcript(meeting_urls[2])

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,1.02,6.02,"And this number of thoughts here must be,",en,None,Dmitriy Grankin,2025-09-03T11:10:49.219371+00:00,2025-09-03T11:10:54.219371+00:00
1,8.15,14.15,"Um, yeah, the thing is that we need to speak a little bit",en,None,Dmitriy Grankin,2025-09-03T11:10:56.349371+00:00,2025-09-03T11:11:02.349371+00:00
2,10.19,11.63,"yeah, the thing is that.",en,None,Dmitriy Grankin,2025-09-03T11:10:58.389371+00:00,2025-09-03T11:10:59.829371+00:00
3,14.15,23.55,"and wait for the third bot to output something, because it did not output anything for now,",en,None,Dmitriy Grankin,2025-09-03T11:11:02.349371+00:00,2025-09-03T11:11:11.749371+00:00
4,23.55,31.55,we are speaking. And it should be fine right now. And it's good.,en,None,Dmitriy Grankin,2025-09-03T11:11:11.749371+00:00,2025-09-03T11:11:19.749371+00:00


In [15]:
running_bots = client.get_running_bots_status()
len(running_bots) #should be 2



DEBUG: Making GET request to http://localhost:18056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 's6SWLYY0wuniAHVIoHpJoYy9ccvdZbdI5xyolkVu'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:11:27 GMT, Wed, 03 Sep 2025 11:11:27 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '871', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[{"container_id":"3c933c2b88656d9412d38a4fa5d1f122291a053de955e1a0fa016d70aa3addfb","container_name":"vexa-bot-238-33ad9e50","platform":"google_meet","native_meeting_id":"wjf-bhmw-bix","status":"Up About a minute","normalized_status":"Up","created_at":"2025-09-03T11:10:13+00:00","labels":{"org.opencontainers.image.ref.name":"ubuntu","org.opencontainers.image.version":"22.04","vexa.user_id":"111"},"meeting_id_from_name":"238"},{"container_id":"63e19181b1aa060c797e9d55cb198e703c912...


2

In [19]:
[m['status'] for m in meetings]

['active', 'completed', 'active']

In [21]:
meetings = client.get_meetings()
[m['status'] for m in meetings] == ['active', 'completed', 'active']


DEBUG: Making GET request to http://localhost:18056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 's6SWLYY0wuniAHVIoHpJoYy9ccvdZbdI5xyolkVu'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:12:52 GMT, Wed, 03 Sep 2025 11:12:52 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1183', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":238,"user_id":111,"platform":"google_meet","native_meeting_id":"wjf-bhmw-bix","constructed_meeting_url":"https://meet.google.com/wjf-bhmw-bix","status":"active","bot_container_id":"vexa-bot-238-33ad9e50","start_time":"2025-09-03T11:10:46.871812","end_time":null,"data":{},"created_at":"2025-09-03T11:10:13.968080","updated_at":"2025-09-03T11:10:46.869764"},{"id":237,"user_id":111,"platform":"google_meet","native_meeting_id":"fjp-cbnn-hug","constructed_meeting_url":"https://meet....


True

In [22]:
for m in meeting_urls:
    try:stop_bot(m)
    except Exception as e:
        print(e)


DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/xdy-txjo-fyh
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 's6SWLYY0wuniAHVIoHpJoYy9ccvdZbdI5xyolkVu'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:13:03 GMT, Wed, 03 Sep 2025 11:13:02 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '59', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted and is being processed."}...

DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/fjp-cbnn-hug
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 's6SWLYY0wuniAHVIoHpJoYy9ccvdZbdI5xyolkVu'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 404
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:13:03 GMT, Wed, 03 Sep 2025 11:13:02 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '38', 'content-type': 'application/json'}
DE

In [23]:
meetings = client.get_meetings()
[m['status'] for m in meetings] == ['completed', 'completed', 'completed']


DEBUG: Making GET request to http://localhost:18056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 's6SWLYY0wuniAHVIoHpJoYy9ccvdZbdI5xyolkVu'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 11:13:14 GMT, Wed, 03 Sep 2025 11:13:14 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1343', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":238,"user_id":111,"platform":"google_meet","native_meeting_id":"wjf-bhmw-bix","constructed_meeting_url":"https://meet.google.com/wjf-bhmw-bix","status":"completed","bot_container_id":"vexa-bot-238-33ad9e50","start_time":"2025-09-03T11:10:46.871812","end_time":"2025-09-03T11:13:05.385445","data":{"languages":["en"],"participants":["Dmitriy Grankin"]},"created_at":"2025-09-03T11:10:13.968080","updated_at":"2025-09-03T11:13:05.390777"},{"id":237,"user_id":111,"platform":"google_m...


True